> Note: structure of GraphQL response is not consistent, these examples may not work in all cases.

In [1]:
import re
import pandas as pd
from twitter.scraper import Scraper
from twitter.util import *

In [ ]:
scraper = Scraper(session=init_session())

### get chat log

In [ ]:
room_id = '1eaJbrAPnBVJX'
spaces = scraper.spaces(rooms=[room_id], audio=0, chat=1)

In [4]:
chat = pd.json_normalize(spaces[0]['chat'])
chat = chat[chat['payload.body.final'] == True]
dates = ['payload.body.timestamp']
chat[dates] = chat[dates].apply(pd.to_datetime, unit='ms').apply(lambda x: x.dt.strftime("%Y-%m-%d %H:%M:%S %z"))
chat = chat.sort_values('payload.body.timestamp').reset_index(drop=True)
chat = chat[[
    'payload.room',
    'payload.body.timestamp',
    'payload.sender.twitter_id',
    'payload.body.username',
    'payload.body.body',
]]
chat.columns = chat.columns.str.replace('(payload|body|sender).','',regex=True).str.replace('.','_')
# chat.to_csv(f'{room_id}.csv',index=False)
chat

,room,timestamp,twitter_id,username,body
0,1eaJbrAPnBVJX,2023-05-25 21:32:19,1106321031566893057,jimfarley98,
1,1eaJbrAPnBVJX,2023-05-25 21:32:55,1106321031566893057,jimfarley98,
2,1eaJbrAPnBVJX,2023-05-25 21:33:01,1106321031566893057,jimfarley98,
3,1eaJbrAPnBVJX,2023-05-25 21:33:13,1106321031566893057,jimfarley98,"Hi Alan, are you there?"
4,1eaJbrAPnBVJX,2023-05-25 21:33:17,44196397,elonmusk,I am.
...,...,...,...,...,...
251,1eaJbrAPnBVJX,2023-05-25 22:00:00,1106321031566893057,jimfarley98,"Yeah, it's a great platform."
252,1eaJbrAPnBVJX,2023-05-25 22:00:03,44196397,elonmusk,"Alright, cool. Sounds good. Thank you."
253,1eaJbrAPnBVJX,2023-05-25 22:00:03,1106321031566893057,jimfarley98,OK. Thank you.
254,1eaJbrAPnBVJX,2023-05-25 22:00:04,44196397,elonmusk,Right.


### query chat

In [5]:
flags = re.I
(chat
 .query('body.str.contains("\sford",regex=True,flags=@flags)')
 .query('username.str.contains("elonmusk",regex=True,flags=@flags)')
)

## alternatively
# chat[
#     chat.body.str.contains('\sford',regex=True,flags=re.I)
#     &
#     chat.username.str.contains('elonmusk',regex=True,flags=re.I)
# ]

,room,timestamp,twitter_id,username,body
22,1eaJbrAPnBVJX,2023-05-25 21:35:05,44196397,elonmusk,"Yeah, well, well, it's certainly super excitin..."
30,1eaJbrAPnBVJX,2023-05-25 21:36:00,44196397,elonmusk,API access so like you know like a Ford vehicl...
33,1eaJbrAPnBVJX,2023-05-25 21:36:23,44196397,elonmusk,"And we're very, very much appreciative of of F..."
47,1eaJbrAPnBVJX,2023-05-25 21:38:29,44196397,elonmusk,But but I think it is the the the teams have d...
191,1eaJbrAPnBVJX,2023-05-25 21:53:52,44196397,elonmusk,"Yeah, I agree with that. We should probably no..."
211,1eaJbrAPnBVJX,2023-05-25 21:56:13,44196397,elonmusk,"So, you know, it is certainly the Tesla intent..."
248,1eaJbrAPnBVJX,2023-05-25 21:59:45,44196397,elonmusk,"Likewise there, it's an honor to be working wi..."
